In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Создадим датасет с данных по клиентам Банка с кредитными картами. Имеются данные:

visits - количество обращений клиента в офис/горячую линию Банка

balance_credit - использованный остаток от овердрафта

percent - ставка по процентам (в год)

day_over - дни, на протяжении которых использовался овердрафт

debt - процентный долг (за пользование овердрафтом) по кредитной карте

In [80]:
# Создаём сэмпл
n_samples = 500

balance_credit = np.random.choice(50000, n_samples) + 1000    #генерируем остатки от овердрафта от 1 до 50 тыс. р.
percent = np.random.choice(20, n_samples) + 5                 #генерируем процентную ставку от 5% до 20%
day_over = np.random.choice(365, n_samples)                   #генерируем дни от 0 до 365  
visits = np.random.choice(15, n_samples)                      #генерируем обращения клиента от 0 до 15

debt = (balance_credit * percent * day_over) / 365 * 0.01     #так в банке высчитывается процентный долг за пользование овердрафтом, но мы этого изначально не якобы не знаем!!!

data = pd.DataFrame({'balance_credit': balance_credit, 'percent': percent, 'visits': visits, 'day_over': day_over, 'debt': debt})
data.head()

,balance_credit,percent,visits,day_over,debt
0,18487,6,3,158,480.155507
1,28652,14,7,194,2132.022795
2,25680,12,11,133,1122.884384
3,28321,20,12,11,170.701918
4,5537,23,8,240,837.376438


Обучим модель на всех признаках balance_credit, visits, percent и day_over

In [81]:
from sklearn.metrics import mean_absolute_error

X = data[['balance_credit', 'visits', 'percent', 'day_over']]
y = data['debt']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['balance_credit', 'visits', 'percent', 'day_over']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [7.67056013e-02 8.23517743e+00 1.35336410e+02 1.08139095e+01]
Bias: -4038.979835166123
Error: 681.6645415221523


Bias и ошибка довольно высокие. Линейной корреляции нет. 

Попробуем обучить модель только на признаках balance_credit и percent

In [82]:
X = data[['balance_credit', 'percent']]
y = data['debt']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['balance_credit', 'percent']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [7.96160261e-02 1.51380549e+02]
Bias: -2258.6592390745554
Error: 1112.3051044555843


Ситуация не изменилась. Модель очень плохо предсказывает debt (процентный долг)

Попробуем создань новый признак, использующий 3 уже имеющихся у нас признака!

In [90]:
# Создаем новый признак
data['duty'] = data['balance_credit'] * data['percent'] * data['day_over'] / 365 
data.head(5)

,balance_credit,percent,visits,day_over,debt,duty
0,18487,6,3,158,480.155507,48015.550685
1,28652,14,7,194,2132.022795,213202.279452
2,25680,12,11,133,1122.884384,112288.438356
3,28321,20,12,11,170.701918,17070.191781
4,5537,23,8,240,837.376438,83737.643836


Обучим модель по новому признаку duty

In [91]:
X = data[['duty']]
y = data['debt']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['duty']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [0.01]
Bias: -2.5011104298755527e-12
Error: 1.8274612934021662e-12


Вес определён абсолютно точно. Сдвиг и ошибка практически равны 0. Соответвенно мы нашли признак (duty), который линейно зависим от debt и использует только 3 признака из нашей базы данных. Линейная модель обучена и она отлично предсказывает параметр debt. Таким образом можно сделать вывод, что в нашей базе данных признак visits (количество обращений клиента) является лишним и не оказывает никакого влияния на предсказание debt (процентный долг за пользование овердрафтом).